In [ ]:
! pip install accelerate peft bitsandbytes trl py7zr auto-gptq optimum transformers

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
import os
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, GPTQConfig
from peft import LoraConfig, AutoPeftModelForCausalLM, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer

In [ ]:
data = load_dataset("samsum", split="train")

In [ ]:
data[0]

{'id': '13818513',
 'dialogue': "Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)",
 'summary': 'Amanda baked cookies and will bring Jerry some tomorrow.'}

In [ ]:
data.to_pandas()

,id,dialogue,summary
0,13818513,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,13681000,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...
3,13730747,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,13728094,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com..."
...,...,...,...
14727,13863028,Romeo: You are on my ‘People you may know’ lis...,Romeo is trying to get Greta to add him to her...
14728,13828570,Theresa: <file_photo>\r\nTheresa: <file_photo>...,Theresa is at work. She gets free food and fre...
14729,13819050,John: Every day some bad news. Japan will hunt...,Japan is going to hunt whales again. Island an...
14730,13828395,Jennifer: Dear Celia! How are you doing?\r\nJe...,Celia couldn't make it to the afternoon with t...


In [ ]:
data.to_pandas().iloc[0]

,0
id,13818513
dialogue,Amanda: I baked cookies. Do you want some?\r\...
summary,Amanda baked cookies and will bring Jerry some...


In [ ]:
data.to_pandas().iloc[0, :]

,0
id,13818513
dialogue,Amanda: I baked cookies. Do you want some?\r\...
summary,Amanda baked cookies and will bring Jerry some...


In [ ]:
data.to_pandas().iloc[:, 1][0]

"Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)"

In [ ]:
data.to_pandas().iloc[:, 2][0]

'Amanda baked cookies and will bring Jerry some tomorrow.'

LLM Input:


###Human: Summarize this following dialoge:

Amanda: I baked  cookies. Do you want some?
Jerry: Sure!
Amanda: I'll bring you tomorrow

###AI Assistant(Model):
Amanda baked cookies and will bring Jerry some tomorrow.


In [ ]:
data_df = data.to_pandas()

In [ ]:
data_df.columns

Index(['id', 'dialogue', 'summary'], dtype='object')

In [ ]:
data_df['text'] = data_df[['dialogue', 'summary']].apply(lambda x: "###Human: Summarize this following dialogue: " + x['dialogue'] + "\n###Assistant: " + x['summary'], axis=1)

In [ ]:
data_df.head()

,id,dialogue,summary,text
0,13818513,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...,###Human: Summarize this following dialogue: A...
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...,###Human: Summarize this following dialogue: O...
2,13681000,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...,###Human: Summarize this following dialogue: T...
3,13730747,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...,###Human: Summarize this following dialogue: E...
4,13728094,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com...",###Human: Summarize this following dialogue: S...


In [ ]:
print(data_df.iloc[0][3])

###Human: Summarize this following dialogue: Amanda: I baked  cookies. Do you want some?
Jerry: Sure!
Amanda: I'll bring you tomorrow :-)
###Assistant: Amanda baked cookies and will bring Jerry some tomorrow.


<ipython-input-15-101905aafc9b>:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(data_df.iloc[0][3])


In [ ]:
data = Dataset.from_pandas(data_df)

In [ ]:
data

Dataset({
    features: ['id', 'dialogue', 'summary', 'text'],
    num_rows: 14732
})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

In [ ]:
tokenizer.pad_token_id

In [ ]:
tokenizer.pad_token

In [ ]:
tokenizer.eos_token_id

2

In [ ]:
tokenizer.eos_token

'</s>'

In [ ]:
tokenizer.pad_token=tokenizer.eos_token

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True, # Enable 4-bit quantization
    bnb_4bit_compute_dtype=torch.float16, # Use fp16 computation
    bnb_4bit_use_double_quant=True,  # Use double quantization for memory efficiency
)

In [ ]:
# We are loading the original model, we are loading quantized model -> using 'quantization_config'

model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    quantization_config=quantization_config,
    device_map="auto"  #it will automatically map with gpu or cpu
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Loading Quantized model (because I faced memory issue)

In [ ]:
## "QUANTIZED GPTQ MODEL WE USED"

# tokenizer = AutoTokenizer.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.1-GPTQ")

# quantization_config=GPTQConfig(bits=4, tokenizer=tokenizer)

# # We are loading the original model, we are loading quantized model -> using 'quantization_config'

# model = AutoModelForCausalLM.from_pretrained(
#     "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ",   # Quantized model
#     quantization_config=quantization_config,
#     device_map="auto"  #it will automatically map with gpu or cpu
# )

In [ ]:
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): Mist

In [ ]:
model = prepare_model_for_kbit_training(model)  # supress your model for kbit (1 bit or 4 or 16 or 32 bit)
# in our case it is 4bit

### This is my "quantized model"

#### load the model into the lower precision

#### for the memory efficient training

In [ ]:
model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): Mist

### LORA

In [ ]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,  # for scaling
    lora_dropout=0.05,  # for reduce overfitting
    bias="none",
    task_type="CAUSAL_LM", # with respect to generation
    target_modules=["q_proj", "v_proj"]  # 'query' and 'value' place a vital role in self attention
)

In [ ]:
model = get_peft_model(model, peft_config)  # Using this model i can perform QLORA SFT

In [ ]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_pro

In [ ]:
training_arguments = TrainingArguments(
    output_dir="mistral-finetuned-samsum",  # saved model name
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,  # back propagation step
    optim="paged_adamw_32bit",     # back propagation step
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    save_strategy="epoch",
    logging_steps=100,
    num_train_epochs=1,  # no of epochs
    fp16=True,
    push_to_hub=True,
    report_to="none"
)

In [ ]:
data

Dataset({
    features: ['id', 'dialogue', 'summary', 'text'],
    num_rows: 14732
})

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=data,
    peft_config=peft_config,
    # tokenizer=tokenizer,
    args=training_arguments,
)

Converting train dataset to ChatML:   0%|          | 0/14732 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/14732 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/14732 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/14732 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
100,1.869500
200,1.758500
300,1.715100


Step,Training Loss
100,1.869500
200,1.758500
300,1.715100


# Inferencing

In [ ]:
# !cp -r <current-location> <drive-location>

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/mistral-finetuned-samsum")

In [ ]:
model = AutoPeftModelForCausalLM.from_pretrained("/content/drive/MyDrive/mistral-finetuned-samsum",
                                                 low_cpu_mem_usage=True,
                                                 return_dict=True,
                                                 torch_dtype=torch.float16,
                                                 device_map="auto")

In [ ]:
inputs = tokenizer("""
###Human: Summarize this following dialogue: Sunny: I'm at the railway station in chennai Karthik: No problems so far? Sunny: no, everythong's going smoothly Karthik: good. lets meet there soon!
###Assistant: """, return_tensors="pt").to("cuda")

In [ ]:
output = model.generate(**inputs, max_new_tokens=100, do_sample=True, top_p=0.9, temperature=0.8)

In [ ]:
tokenizer.decode(output[0], skip_special_tokens=True)